
# CAMx高空點源排放檔案之產生

## 背景
- 此處處理TEDS PM/VOCs年排放量之劃分、與時變係數相乘、整併到光化模式網格系統內。
- 高空點源的**時變係數**檔案需按CEMS數據先行[展開](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/ptseE_ONS/)。
- 排放量整體處理原則參見[處理程序總綱](https://sinotec2.github.io/Focus-on-Air-Quality/EmsProc/#處理程序總綱)、針對[點源之處理](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/)及[龐大`.dbf`檔案之讀取](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/dbf2csv.py/)，為此處之前處理。程式也會呼叫到[ptse_sub](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/ptse_sub/)中的副程式

## 副程式說明

### [ptse_sub](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/ptse_sub/)

### 對煙道座標進行叢集整併
如題所示。整併的理由有幾：
- 鄰近煙流在近距離重疊後，會因整體煙流熱量的提升而提升其最終煙流高度，從而降低對地面的影響，此一現象並未在模式的煙流次網格模式中予以考量，須在模式外先行處理。
- 重複計算較小煙流對濃度沒有太大的影響，卻耗費大量儲存、處理的電腦資源，因此整併有其必須性。
  - 即使將較小的點源按高度切割併入面源，還是保留此一機制，以避免點源個數無限制擴張，有可能放大因品質不佳造成奇異性。

#### cluster_xy
- 調用`sklearn`之`KMeans`來做叢集分析



In [1]:
kuang@node03 /nas1/TEDS/teds11/ptse
$ cat -n cluster_xy.py
def cluster_xy(df,C_NO):
  from sklearn.cluster import KMeans
  from pandas import DataFrame
  import numpy as np
  import sys


- 由資料庫中選擇同一**管編**之煙道出來




In [2]:
  b=df.loc[df.CP_NO.map(lambda x:x[:8]==C_NO)]
  n=len(b)
  if n==0:sys.exit('fail filtering of '+C_NO)
  colb=b.columns


- 此**管編**所有煙道的座標及高度，整併為一個大矩陣
  - 進行KMeans叢集分析




In [3]:
  x=[b.XY[i][0] for i in b.index]
  y=[b.XY[i][1] for i in b.index]
  z=b.HEI
  M=np.array([x, y, z]).T
  clt = KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300, \
       n_clusters=10, n_init=10, n_jobs=-1, precompute_distances='auto', \
       random_state=None, tol=0.0001, verbose=0)
  kmeans=clt.fit(M)


- 放棄原來的座標，改採叢集的平均位置




In [4]:
#  np.array(clt.cluster_centers_) for group
  b_lab=np.array(clt.labels_)
  df.loc[b.index,'UTM_E']=[np.array(clt.cluster_centers_)[i][0] for i in b_lab]
  df.loc[b.index,'UTM_N']=[np.array(clt.cluster_centers_)[i][1] for i in b_lab]
  return df


#### XY_pivot
- 工廠點源個數太多者(如中鋼)，在座標叢集化之前，以pivot_tab取其管道(**管煙**編號=**管編**+**煙編**)排放量之加總值
  - 調用模組

   


In [5]:
#XY clustering in CSC before pivotting
#df=cluster_xy(df,'E5600841')
#pivotting
def XY_pivot(df,col_id,col_em,col_mn,col_mx):
  from pandas import pivot_table,merge
  import numpy as np


- 3類不同屬性欄位適用不同的`aggfunc`
  - 排放量(`col_em`)：加總
  - 煙道高度(`col_mx`)：最大值
  - 煙道其他參數(`col_mn`)：平均

   


In [6]:
  df_pv1=pivot_table(df,index=col_id,values=col_em,aggfunc=np.sum).reset_index()
  df_pv2=pivot_table(df,index=col_id,values=col_mn,aggfunc=np.mean).reset_index()
  df_pv3=pivot_table(df,index=col_id,values=col_mx,aggfunc=max).reset_index()


- 整併、求取等似直徑、以使流量能守恒

   


In [7]:
  df1=merge(df_pv1,df_pv2,on=col_id)
  df=merge(df1,df_pv3,on=col_id)
  df['DIA']=[np.sqrt(4/3.14159*q/60*(t+273)/273/v) for q,t,v in zip(df.ORI_QU1,df.TEMP,df.VEL)]
  return df



## 主程式說明

### 程式之執行
- 此處按月執行。由於nc檔案時間展開後，檔案延長非常緩慢，拆分成主程式（`ptseE.py`）與輸出程式（`wrtE.py`）二段進行。




In [8]:
for m in 0{1..9} 1{0..2};do python ptseE.py 19$m;done
for m in 0{1..9} 1{0..2};do python wrtE.py 19$m;done



### 程式基本定義、資料庫檔案QC、nc檔案之延展
- 調用模組
  - 因無另存處理過後的資料庫，因此程式還是會用到[ptse_sub](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/ptse_sub/)中的副程式`CORRECT`, `add_PMS`, `check_nan`, `check_landsea`, `FillNan`, `WGS_TWD`, `Elev_YPM`

   


In [1]:
#kuang@node03 /nas1/TEDS/teds11/ptse
#$ cat -n ptseE.py
#! crding = utf8
from pandas import *
import numpy as np
import os, sys, subprocess
import netCDF4
import twd97
import datetime
from calendar import monthrange
from scipy.io import FortranFile
from mostfreqword import mostfreqword
from ptse_sub import CORRECT, add_PMS, check_nan, check_landsea, FillNan, WGS_TWD, Elev_YPM
from ioapi_dates import jul2dt, dt2jul
from cluster_xy import cluster_xy, XY_pivot


- 程式相依性及年月定義(由引數)
  - `pncgen`、`ncks`是在`wrtE.py`階段使用

   


In [5]:
#Main
#locate the programs and root directory
pncg=subprocess.check_output('which pncgen',shell=True).decode('utf8').strip('\n')
ncks=subprocess.check_output('which ncks',shell=True).decode('utf8').strip('\n')
hmp=subprocess.check_output('pwd',shell=True).decode('utf8').strip('\n').split('/')[1]
P='./'
#time and space initiates
ym='1901' #sys.argv[1]
mm=ym[2:4]
mo=int(mm)
yr=2000+int(ym[:2]);TEDS='teds'+str(int((yr-2016)/3)+10)


- 使用`Hs`進行篩選「高空」點源

   


In [3]:
Hs=10 #cutting height of stacks


- 起迄日期、模擬範圍中心點位置
 
   


In [4]:
ntm=(monthrange(yr,mo)[1]+2)*24+1
bdate=datetime.datetime(yr,mo,1)+datetime.timedelta(days=-1+8./24)
edate=bdate+datetime.timedelta(days=ntm/24)#monthrange(yr,mo)[1]+3)
Latitude_Pole, Longitude_Pole = 23.61000, 120.9900
Xcent, Ycent = twd97.fromwgs84(Latitude_Pole, Longitude_Pole)


- nc模版的應用與延展、注意`name`在新版`NCF`可能會被保留不能更改。(另在`CAMx`程式碼中處理)

   


In [7]:
#prepare the uamiv template
print('template applied')
NCfname='fortBE.413_'+TEDS+'.ptsE'+mm+'.nc'
try:
  nc = netCDF4.Dataset(NCfname, 'r+')
except:
  os.system('cp '+P+'template_v7.nc '+NCfname)
  nc = netCDF4.Dataset(NCfname, 'r+')
V=[list(filter(lambda x:nc.variables[x].ndim==j, [i for i in nc.variables])) for j in [1,2,3,4]]
nt,nv,dt=nc.variables[V[2][0]].shape
nv=len([i for i in V[1] if i !='CP_NO'])
nc.SDATE,nc.STIME=dt2jul(bdate)
nc.EDATE,nc.ETIME=dt2jul(edate)
nc.NOTE='Point Emission'
nc.NOTE=nc.NOTE+(60-len(nc.NOTE))*' '
nc.NVARS=nv
#Name-names may encounter conflicts with newer versions of NCFs and PseudoNetCDFs.
#nc.name='PTSOURCE  '
nc.NSTEPS=ntm
if 'ETFLAG' not in V[2]:
  zz=nc.createVariable('ETFLAG',"i4",("TSTEP","VAR","DATE-TIME"))
if nt!=ntm or (nc.variables['TFLAG'][0,0,0]!=nc.SDATE and nc.variables['TFLAG'][0,0,1]!=nc.STIME):
  for t in range(ntm):
    sdate,stime=dt2jul(bdate+datetime.timedelta(days=t/24.))
    nc.variables['TFLAG'][t,:,0]=[sdate for i in range(nv)]
    nc.variables['TFLAG'][t,:,1]=[stime for i in range(nv)]
    ndate,ntime=dt2jul(bdate+datetime.timedelta(days=(t+1)/24.))
    nc.variables['ETFLAG'][t,:,0]=[ndate for i in range(nv)]
    nc.variables['ETFLAG'][t,:,1]=[ntime for i in range(nv)]
nc.close()
#template OK


template applied


- 污染物名稱對照、變數群組定義

   


In [8]:
#item sets definitions
c2s={'NMHC':'NMHC','SOX':'SO2','NOX':'NO2','CO':'CO','PM':'PM'}
c2m={'SOX':64,'NOX':46,'CO':28,'PM':1}
cole=[i+'_EMI' for i in c2s]+['PM25_EMI']
XYHDTV=['UTM_E','UTM_N','HEI','DIA','TEMP','VEL']
colT=['HD1','DY1','HY1']
colc=['CCRS','FCRS','CPRM','FPRM']


- 讀取點源資料庫並進行品質管控。新版`coding`只接受`big5`

   


In [67]:
#Input the TEDS csv file
try:
  df = read_csv('point.csv', encoding='big5')
except:
  df = read_csv('point.csv')
df = check_nan(df)
df = check_landsea(df)
df = WGS_TWD(df)
df = Elev_YPM(df)
#only P??? an re tak einto account
boo=(df.HEI>=Hs) & (df.NO_S.map(lambda x:x[0]=='P'))
df=df.loc[boo].reset_index(drop=True)
#delete the zero emission sources
df['SUM']=[i+j+k+l+m for i,j,k,l,m in zip(df.SOX_EMI,df.NOX_EMI,df.CO_EMI,df.PM_EMI,df.NMHC_EMI)]
df=df.loc[df.SUM>0].reset_index(drop=True)
df['DY1']=[i*j for i,j in zip(df.DW1,df.WY1)]
df['HY1']=[i*j for i,j in zip(df.HD1,df.DY1)]
df=CORRECT(df)
df['CP_NO'] = [i + j for i, j in zip(list(df['C_NO']), list(df['NO_S']))]
#


/opt/miniconda3/envs/geocat/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (54) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


set()
TSP_EMI 0
PM_EMI 0
PM25_EMI 0
SOX_EMI 0
NOX_EMI 0
THC_EMI 0
NMHC_EMI 0
CO_EMI 0
PB_EMI 0


/nas1/TEDS/teds11/ptse/ptse_sub.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if len(idx[0])>0:dia[idx[0]]=he[idx[0]]*0.05
/nas1/TEDS/teds11/ptse/ptse_sub.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[idx[0]]=[t_intv[i] for i in he_i]


- 座標轉換

   


In [68]:
#Coordinate translation
df.UTM_E=df.UTM_E-Xcent
df.UTM_N=df.UTM_N-Ycent
df.SCC=[str(int(i)) for i in df.SCC]
df.loc[df.SCC=='0','SCC']='0'*10


- 對**管煙**編號進行資料庫重整
  - 3類不同屬性欄位適用不同的`aggfunc`
    - 排放量(`col_em`)：加總
    - 煙道高度(`col_mx`)：最大值
    - 煙道其他參數(`col_mn`)：平均

   


In [69]:
#pivot table along the dimension of NO_S (P???)
df_cp=pivot_table(df,index='CP_NO',values=cole+['ORI_QU1'],aggfunc=sum).reset_index()
df_xy=pivot_table(df,index='CP_NO',values=XYHDTV+colT,aggfunc=np.mean).reset_index()
df_sc=pivot_table(df,index='CP_NO',values='SCC', aggfunc=mostfreqword).reset_index()
df1=merge(df_cp,df_xy,on='CP_NO')
df=merge(df1,df_sc,on='CP_NO')
df.head()#debugging

,CP_NO,CO_EMI,NMHC_EMI,NOX_EMI,ORI_QU1,PM25_EMI,PM_EMI,SOX_EMI,DIA,DY1,HD1,HEI,HY1,TEMP,UTM_E,UTM_N,VEL,SCC
0,A3400047P002,0.740,0.0,1.119,189.2,0.023,0.025,0.000,1.10,364.0,15.0,51.0,5460.0,202.0,54060.527015,154494.049724,9.1,10200602
1,A3400109P001,0.318,0.0,0.377,30.0,0.010,0.010,0.000,0.95,364.0,24.0,35.0,8736.0,120.0,54325.725374,154014.654113,7.3,10200602
2,A3400118P001,0.082,0.0,0.097,30.0,0.003,0.003,0.000,0.50,364.0,24.0,53.0,8736.0,150.0,54179.197179,155026.404876,9.1,10200603
3,A3400172P001,0.727,0.0,0.839,3.0,0.023,0.024,0.000,0.46,364.0,24.0,76.0,8736.0,150.0,54200.082535,155161.330165,9.1,10200602
4,A34A1392P001,0.249,0.0,0.557,45.0,0.024,0.025,0.031,0.92,364.0,24.0,57.0,8736.0,150.0,54383.208009,154491.108945,9.1,10200602


- 排放量單位轉換
  - 因小時數在`ons`中已經應用了(個別煙道加總為1.)，因此只需考量重量之轉換即可。
  - 留下之前計算版本，以供對照

   


In [70]:
#T/year to g/hour
for c in cole:
  df[c]=[i*1E6 for i in df[c]]
#  df[c]=[i*1E6/j/k for i,j,k in zip(df[c],df.DY1,df.HD1)]


- CAMx版本差異在此選擇
  - 6~7版的差異可以參考[CMAQ/CAMx排放量檔案之轉換](http://www.evernote.com/l/AH1z_n2U--lM-poNlQnghsjFBfDEY6FalgM/)，
  - 如要做不同版本，應重新準備模版階段

   


In [16]:
#determination of camx version
ver=7
if 'XSTK' in V[0]:ver=6
print('NMHC/PM splitting and expanding')


NMHC/PM splitting and expanding



### VOCs與PM的劃分
- 讀取profile number、preference table

   


In [17]:
print('NMHC/PM splitting and expanding')
#prepare the profile and CBMs
fname='/'+hmp+'/SMOKE4.5/data/ge_dat/gsref.cmaq_cb05_soa.txt'
gsref=read_csv(fname,delimiter=';',header=None)
col='SCC Speciation_profile_number Pollutant_ID'.split()+['C'+str(i) for i in range(3,10)]
gsref.columns=col
for c in col[3:]:
  del gsref[c]
fname='/'+hmp+'/SMOKE4.5/data/ge_dat/gspro.cmaq_cb05_soa.txt'
gspro=read_csv(fname,delimiter=';',header=None)
col=['Speciation_profile_number','Pollutant_ID','Species_ID','Split_factor','Divisor','Mass_Fraction']
gspro.columns=col
gspro.head() #debugging

NMHC/PM splitting and expanding


,Speciation_profile_number,Pollutant_ID,Species_ID,Split_factor,Divisor,Mass_Fraction
0,B09C4,ISOP,ISOP,1.0,60.0,1.1333
1,B09C4,MONO,TERPB,1.0,120.0,1.1333
2,B09C4,MONO,OLE,0.5,120.0,0.1133
3,B09C4,MONO,PAR,6.0,120.0,0.6800
4,B09C4,MONO,ALD2,1.5,120.0,0.3400


- 自從TEDS9之後，國內新增(或修正)很多資料庫中沒有的SCC。此處以對照既有SCC碼簡單處理。
  - 對照方式：網站上找到最新的SCC資料庫，找到新SCC的製程類別特性
  - 找到既有SCC profile number資料表中，前幾碼(4~6)相同的SCC，如果類似就指定取代
  - 如果真的找不到，也只能找數字最接近的取代

   


In [71]:
#new SCC since TEDS9,erase and substude
sccMap={
'30111103':'30111199', #not in df_scc2
'30112401':'30112403', #Industrial Processes  Chemical Manufacturing  Chloroprene Chlorination Reactor
'30115606':'30115607',#Industrial Processes  Chemical Manufacturing  Cumene  Aluminum Chloride Catalyst Process: DIPB Strip
'30118110':'30118109',#Industrial Processes  Chemical Manufacturing  Toluene Diisocyanate  Residue Vacuum Distillation
'30120554':'30120553', #not known, 548~  Propylene Oxide Mixed Hydrocarbon Wash-Decant System Vent
'30117410':'30117421',
'30117411':'30117421',
'30117614':'30117612',
'30121125':'30121104',
'30201111':'30201121',
'30300508':'30300615',
'30301024':'30301014',
'30400213':'30400237',
'30120543':'30120502',
'40300215':'40300212'} #not known
for s in sccMap:
  df.loc[df.SCC==s,'SCC']=sccMap[s]   


- 只篩選有關的SCC以縮減資料庫長度、提升對照速度
  - 因有些profile number含有英文字，在此先做整理，以使格式一致

   


In [20]:
#reduce gsref and gspro
dfV=df.loc[df.NMHC_EMI>0].reset_index(drop=True)
gsrefV=gsref.loc[gsref.SCC.map(lambda x:x in set(dfV.SCC))].reset_index(drop=True)
prof_alph=set([i for i in set(gsrefV.Speciation_profile_number) if i.isalpha()])
gsrefV=gsrefV.loc[gsrefV.Speciation_profile_number.map(lambda x:x not in prof_alph)].reset_index(drop=True)
gsproV=gspro.loc[gspro.Speciation_profile_number.map(lambda x:x in set(gsrefV.Speciation_profile_number))].reset_index(drop=True)
set(gsproV.Species_ID) #debugging

{'ALD2',
 'ALDX',
 'BENZ',
 'CH4',
 'CL2',
 'CO',
 'CPRM',
 'ETH',
 'ETHA',
 'ETOH',
 'FORM',
 'FPRM',
 'HCL',
 'HFLUX',
 'HGIIGAS',
 'HGNRVA',
 'IOLE',
 'ISOP',
 'MEOH',
 'NH3',
 'NO',
 'NOX',
 'NR',
 'OLE',
 'PAR',
 'PEC',
 'PHGI',
 'PNO3',
 'POA',
 'PSO4',
 'SO2',
 'SULF',
 'TERP',
 'TOL',
 'UNR',
 'XYL'}

- 只篩選有關的profile number且污染物含有`TOG`者

   


In [21]:
pp=[]
for p in set(gspro.Speciation_profile_number):
  a=gsproV.loc[gsproV.Speciation_profile_number==p]
  if 'TOG' not in set(a.Pollutant_ID):pp.append(p)
boo=(gspro.Speciation_profile_number.map(lambda x:x not in pp)) & (gspro.Pollutant_ID=='TOG')
gsproV=gspro.loc[boo].reset_index(drop=True)
set(gsproV.Species_ID) #debugging

{'ALD2',
 'ALDX',
 'BENZ',
 'CH4',
 'ETH',
 'ETHA',
 'ETOH',
 'FORM',
 'IOLE',
 'ISOP',
 'MEOH',
 'OLE',
 'PAR',
 'POA',
 'TERP',
 'TOL',
 'UNR',
 'XYL'}

- 準備乘數矩陣，其大小為`(SCC總數,CBM物質項目總數)`

   


In [22]:
cbm=list(set([i for i in set(gsproV.Species_ID) if i in V[1]]))
idx=gsproV.loc[gsproV.Species_ID.map(lambda x:x in cbm)].index
sccV=list(set(dfV.SCC))
sccV.sort()
nscc=len(sccV)
prod=np.zeros(shape=(nscc,len(cbm)))


- 對得到SCC，但是資料庫中卻沒有`TOG`也沒有`VOC`。記下、執行下個SCC

   


In [24]:
#dfV but with PM scc(no TOG/VOC in gspro), modify those SCC to '0'*10 in dfV, drop the pro_no in gsproV
noTOG_scc=[]
for i in range(nscc):
  s=sccV[i]
  p=list(gsrefV.loc[gsrefV.SCC==s,'Speciation_profile_number'])[0]
  a=gsproV.loc[gsproV.Speciation_profile_number==p]
  if 'TOG' not in set(a.Pollutant_ID) and 'VOC' not in set(a.Pollutant_ID):
    noTOG_scc.append(s)
    continue
len(noTOG_scc),noTOG_scc[:5] #debugging

(1, ['10301303'])

- 找到對應的profile number，將分率存入乘數矩陣`prod`

   


In [29]:
for i in range(nscc):
  if 'TOG' not in set(a.Pollutant_ID) and 'VOC' not in set(a.Pollutant_ID): continue
  boo=(gsproV.Speciation_profile_number==p) & (gsproV.Pollutant_ID=='TOG')
  a=gsproV.loc[boo]
  for c in a.Species_ID:
    if c not in cbm:continue
    j=cbm.index(c)
    f=a.loc[a.Species_ID==c,'Mass_Fraction']
    d=a.loc[a.Species_ID==c,'Divisor']
    prod[i,j]+=f/d
print(sccV[5])
print([(cbm[i],prod[5,i]) for i in range(len(cbm))]) #debugging

10100601
[('FORM', 0.0), ('TOL', 0.0), ('ETHA', 0.003591739000299312), ('TERP', 0.0), ('ETH', 0.01240500192491409), ('ALD2', 0.0), ('OLE', 0.0004752826771990361), ('MEOH', 0.0), ('BENZ', 0.00394306622046861), ('XYL', 0.0), ('ETOH', 0.0), ('ISOP', 0.0), ('ALDX', 0.0), ('POA', 0.0), ('IOLE', 0.0), ('PAR', 0.004418340684538093)]


- `NMHC_EMI`排放量乘上乘數矩陣，形成`CBM`排放量

   


In [72]:
df.loc[df.SCC.map(lambda x:x in noTOG_scc),'SCC']='0'*10
for c in cbm:
  df[c]=0.
for s in set(dfV.SCC):
  i=sccV.index(s)
  idx=df.loc[df.SCC==s].index
  for c in cbm:
    j=cbm.index(c)
    df.loc[idx,c]=[prod[i,j]*k for k in df.loc[idx,'NMHC_EMI']]
df.loc[(df.SCC==sccV[5])&(df.NMHC_EMI>0)].head() #debugging

,CP_NO,CO_EMI,NMHC_EMI,NOX_EMI,ORI_QU1,PM25_EMI,PM_EMI,SOX_EMI,DIA,DY1,...,OLE,MEOH,BENZ,XYL,ETOH,ISOP,ALDX,POA,IOLE,PAR
12354,L9202099P204,1000.0,1000.0,1000.0,15.0,0.0,0.0,0.0,0.7,357.0,...,0.475283,0.0,3.943066,0.0,0.0,0.0,0.0,0.0,0.0,4.418341
12355,L9202099P205,1000.0,1000.0,1000.0,15.0,0.0,0.0,0.0,0.7,357.0,...,0.475283,0.0,3.943066,0.0,0.0,0.0,0.0,0.0,0.0,4.418341


- PM的劃分，詳見[ptse_sub](https://sinotec2.github.io/Focus-on-Air-Quality/EmisProc/ptse/ptse_sub/#%E7%B0%A1%E5%96%AE%E7%9A%84pm%E5%8A%83%E5%88%86%E5%89%AF%E7%A8%8B%E5%BC%8F)

   


In [74]:
#PM splitting
df=add_PMS(df)
df.loc[0]

CP_NO        A3400047P002
CO_EMI           740000.0
NMHC_EMI              0.0
NOX_EMI         1119000.0
ORI_QU1             189.2
PM25_EMI          23000.0
PM_EMI            25000.0
SOX_EMI               0.0
DIA                   1.1
DY1                 364.0
HD1                  15.0
HEI                  51.0
HY1                5460.0
TEMP                202.0
UTM_E        54060.527015
UTM_N       154494.049724
VEL                   9.1
SCC              10200602
FORM                  0.0
TOL                   0.0
ETHA                  0.0
TERP                  0.0
ETH                   0.0
ALD2                  0.0
OLE                   0.0
MEOH                  0.0
BENZ                  0.0
XYL                   0.0
ETOH                  0.0
ISOP                  0.0
ALDX                  0.0
POA                   0.0
IOLE                  0.0
PAR                   0.0
CCRS                  0.0
FCRS                  0.0
CPRM               2000.0
FPRM              23000.0
Name: 0, dty


### 年均排放量乘上時變係數
- **時變係數**檔案，因筆數與不同的定義高度有關，須先執行後，將檔名寫在程式內。
  - 此處的`fns0`\~`fns30`分別是高度0\~30所對應的**時變係數**檔案
  - 好處是：如此可以將對照關係記錄下來
  - 壞處是：必須手動修改程式碼、每版teds的檔案名稱會不一樣

   


In [35]:
#pivot along the axis of XY coordinates
#def. of columns and dicts
fns0={
'CO'  :'CO_ECP7496_MDH8760_ONS.bin',
'NMHC':'NMHC_ECP2697_MDH8760_ONS.bin',
'NOX' :'NOX_ECP13706_MDH8760_ONS.bin',
'PM'  :'PM_ECP17835_MDH8760_ONS.bin',
'SOX' :'SOX_ECP8501_MDH8760_ONS.bin'}
fns10={
'CO'  :'CO_ECP4919_MDH8784_ONS.bin',
'NMHC':'NMHC_ECP3549_MDH8784_ONS.bin',
'NOX' :'NOX_ECP9598_MDH8784_ONS.bin',
'PM'  :'PM_ECP11052_MDH8784_ONS.bin',
'SOX' :'SOX_ECP7044_MDH8784_ONS.bin'}
fns30={
'CO'  :'CO_ECP1077_MDH8784_ONS.bin',
'NMHC':'NMHC_ECP1034_MDH8784_ONS.bin',
'NOX' :'NOX_ECP1905_MDH8784_ONS.bin',
'PM'  :'PM_ECP2155_MDH8784_ONS.bin',
'SOX' :'SOX_ECP1468_MDH8784_ONS.bin'}
F={0:fns0,10:fns10,30:fns30}
fns=F[Hs]


- 排放名稱與空品名稱對照表、排放名稱與分子量對照表

   


In [75]:
cols={i:[c2s[i]] for i in c2s}
cols.update({'NMHC':cbm,'PM':colc})
colp={c2s[i]:i+'_EMI' for i in fns}
colp.update({i:i for i in cbm+colc})
lspec=[i for i in list(colp) if i not in ['NMHC','PM']]
c2m={i:1 for i in colp}
c2m.update({'SO2':64,'NO2':46,'CO':28})
col_id=["C_NO","XY"]
col_em=list(colp.values())
col_mn=['TEMP','VEL','UTM_E', 'UTM_N','HY1','HD1','DY1']
col_mx=['HEI']
df['XY']=[(x,y) for x,y in zip(df.UTM_E,df.UTM_N)]
df["C_NO"]=[x[:8] for x in df.CP_NO]


- 讀取**時變係數**並將其常態化(時間軸加總為1.0)
  - SPECa為每煙道逐時排放量
  
   


In [37]:
print('Time fraction multiplying and expanding')
#matching of the bin filenames
nopts=len(df)
SPECa=np.zeros(shape=(ntm,nopts,len(lspec)))
id365=365
if yr%4==0:id365=366


Time fraction multiplying and expanding


  - 依序開啟各污染物之**時變係數**檔案

   


In [50]:
for spe in list(fns)[:1]: #read the first one for eg.
  fnameO=fns[spe]
  with FortranFile(fnameO, 'r') as f:
    cp = f.read_record(dtype=np.dtype('U12'))
    mdh = f.read_record(dtype=np.int)
    ons = f.read_record(dtype=float)
print(cp[:5],mdh[:5]) #debugging

/tmp/ipykernel_19354/1135795365.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mdh = f.read_record(dtype=np.int)


['A3400047P002' 'A3400109P001' 'A3400118P001' 'A3400172P001'
 'A34A1392P001'] [10100 10101 10102 10103 10104]


  - `FortranFile`的特色是只有總長度，沒有形狀，需要`reshape`
  - 對時間軸加總

   


In [51]:
  ons=ons.reshape(len(cp),len(mdh))
  s_ons=np.sum(ons,axis=1)
  print(ons[:5,:5]) #debugging

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


  - 重新整理順序，只處理有排放的煙道(加總>0者)、且**管煙**編號存在於資料庫(確認用)

   


In [52]:
  #only those CP with emission accounts
  idx=np.where(s_ons>0)
  cp1 = [i for i in cp[idx[0]] if i in list(df.CP_NO)]
  len(idx[0]) #debugging

7496

  - 因為序列的`index`指令會很耗時，先將做成`array`。

   


In [53]:
  idx= np.array([list(cp).index(i) for i in cp1])
  cp, ons, s_ons =cp1,ons[idx,:],s_ons[idx]
  #normalize to be the fractions in a year
  len(idx) #debugging

7461

  - 常態化

   


In [54]:
  ons=ons/s_ons[:,None]


- 製作煙道、當月啟始及終結時間的標籤
  - 從全年的**時變**係數矩陣中提取當月部分，存成`ons2`矩陣

   


In [57]:
  idx_cp=[list(df.CP_NO).index(i) for i in cp]
  ibdate=list(mdh).index(int(bdate.strftime('%m%d%H')))
  iedate=list(mdh).index(int(edate.strftime('%m%d%H')))
  ons2=np.zeros(shape=(nopts,ntm)) #time fractions for this month
  if ibdate>iedate:
    endp=id365*24-ibdate
    ons2[idx_cp,:endp]=ons[:,ibdate:]
    ons2[idx_cp,endp:ntm]=ons[:,:iedate]
  else:
    ons2[idx_cp,:]=ons[:,ibdate:iedate]
  print(cp[:5]) #debugging
  print(s_ons[:5]) #debugging
  print(ons2[:5,:5]) #debugging

['A3400047P002', 'A3400109P001', 'A3400118P001', 'A3400172P001', 'A34A1392P001']
[5460. 8736. 8736. 8736. 8736.]
[[0.00018315 0.00018315 0.00018315 0.00018315 0.00018315]
 [0.00011447 0.00011447 0.00011447 0.00011447 0.00011447]
 [0.00011447 0.00011447 0.00011447 0.00011447 0.00011447]
 [0.00011447 0.00011447 0.00011447 0.00011447 0.00011447]
 [0.00011447 0.00011447 0.00011447 0.00011447 0.00011447]]


- `SPEC`為全年排放總量(時間軸為定值)，單位轉成gmole，或g。

   


In [58]:
  NREC,NC=nopts,len(cols[spe])
  ons =np.zeros(shape=(ntm,NREC,NC))
  SPEC=np.zeros(shape=(ntm,NREC,NC))
  for c in cols[spe]:
    ic=cols[spe].index(c)
    for t in range(ntm):
      SPEC[t,:,ic]=df[colp[c]]/c2m[c]


- 借用之前用過的`ons`矩陣來儲存ons2的轉置結果。並進行排放總量與**時變**係數相乘

   


In [59]:
  OT=ons2.T[:,:]
  for ic in range(NC):
    ons[:,:,ic]=OT
  #whole matrix production is faster than idx_cp selectively manupilated
  for c in cols[spe]:
    if c not in V[1]:continue
    ic=cols[spe].index(c)
    icp=lspec.index(c)
    SPECa[:,:,icp]=SPEC[:,:,ic]*ons[:,:,ic]



### 
- 形成新的逐時資料表(`dfT`)
  - **管煙**編號序列重新整理，取得順位標籤`CP_NOi`
  - 時間標籤`idatetime`

   


In [76]:
print('pivoting along the C_NO axis')
#forming the DataFrame
CPlist=list(set(df.CP_NO))
CPlist.sort()
pwrt=int(np.log10(len(CPlist))+1)
CPdict={i:CPlist.index(i) for i in CPlist}
df['CP_NOi']=[CPdict[i] for i in df.CP_NO]
idatetime=np.array([i for i in range(ntm) for j in range(nopts)],dtype=int)
dfT=DataFrame({'idatetime':idatetime})
print(pwrt) #debugging

pivoting along the C_NO axis
5


- 將原來的排放資料表`df`展開成`dft`

   


In [77]:
ctmp=np.zeros(shape=(ntm*nopts))
for c in col_mn+col_mx+['CP_NOi']+['ORI_QU1']:
  clst=np.array(list(df[c]))
  for t in range(ntm):
    t1,t2=t*nopts,(t+1)*nopts
    a=clst
    if c=='CP_NOi':a=t*10**(pwrt)+clst
    ctmp[t1:t2]=a
  dfT[c]=ctmp


- 將排放量矩陣壓平後，覆蓋原本的資料庫`df`內容。
   


In [78]:
#dfT.C_NOi=np.array(dfT.C_NOi,dtype=int)
for c in lspec:
  icp=lspec.index(c)
  dfT[c]=SPECa[:,:,icp].flatten()
#usage: orig df, index, sum_cols, mean_cols, max_cols
df=XY_pivot(dfT,['CP_NOi'],lspec,col_mn+['ORI_QU1'],col_mx).reset_index()
df['CP_NO']=[int(j)%10**pwrt for j in df.CP_NOi]
print(df.head()) #debugging
df.tail() #debugging

   index  CP_NOi  ALD2  ALDX  BENZ  CCRS            CO  CPRM  ETH  ETHA  ...  \
0      0     0.0   0.0   0.0   0.0   0.0  4.840398e+06   0.0  0.0   0.0  ...   
1      1     1.0   0.0   0.0   0.0   0.0  1.300039e+06   0.0  0.0   0.0  ...   
2      2     2.0   0.0   0.0   0.0   0.0  3.352302e+05   0.0  0.0   0.0  ...   
3      3     3.0   0.0   0.0   0.0   0.0  2.972102e+06   0.0  0.0   0.0  ...   
4      4     4.0   0.0   0.0   0.0   0.0  1.017955e+06   0.0  0.0   0.0  ...   

    HD1     HY1  ORI_QU1   TEMP         UTM_E          UTM_N  VEL   HEI  \
0  15.0  5460.0    189.2  202.0  54060.527015  154494.049724  9.1  51.0   
1  24.0  8736.0     30.0  120.0  54325.725374  154014.654113  7.3  35.0   
2  24.0  8736.0     30.0  150.0  54179.197179  155026.404876  9.1  53.0   
3  24.0  8736.0      3.0  150.0  54200.082535  155161.330165  9.1  76.0   
4  24.0  8736.0     45.0  150.0  54383.208009  154491.108945  9.1  57.0   

        DIA  CP_NO  
0  0.876163      0  
1  0.354318      1  
2  0.

,index,CP_NOi,ALD2,ALDX,BENZ,CCRS,CO,CPRM,ETH,ETHA,...,HD1,HY1,ORI_QU1,TEMP,UTM_E,UTM_N,VEL,HEI,DIA,CP_NO
17847253,17847253,79222501.0,0.0,0.0,0.0,0.0,1.185570e+05,0.0,0.0,0.0,...,24.0,8736.0,30.0,171.0,-47463.012481,295344.775743,6.5,10.0,0.399111,22501
17847254,17847254,79222502.0,0.0,0.0,0.0,0.0,1.185570e+05,0.0,0.0,0.0,...,24.0,8736.0,30.0,171.0,-47453.408814,295334.035409,6.5,10.0,0.399111,22502
17847255,17847255,79222503.0,0.0,0.0,0.0,0.0,9.247449e+06,0.0,0.0,0.0,...,24.0,8736.0,30.0,120.0,-47482.377157,295323.436803,7.3,14.0,0.354318,22503
17847256,17847256,79222504.0,0.0,0.0,0.0,0.0,2.081562e+05,0.0,0.0,0.0,...,24.0,8064.0,15.0,171.0,-48300.319811,295526.653788,6.5,8.0,0.282214,22504
17847257,17847257,79222505.0,0.0,0.0,0.0,0.0,9.300595e+04,0.0,0.0,0.0,...,24.0,8064.0,15.0,171.0,-48300.319811,295526.653788,6.5,9.0,0.282214,22505


- 進行pivot_table整併

   


In [79]:
pv=XY_pivot(df,['CP_NO'],lspec,col_mn+['ORI_QU1'],col_mx).reset_index()
Bdict={CPdict[j]:[bytes(i,encoding='utf-8') for i in j] for j in CPlist}
pv['CP_NOb'] =[Bdict[i] for i in pv.CP_NO]
nopts=len(set(pv))
pv.head()

,index,CP_NO,ALD2,ALDX,BENZ,CCRS,CO,CPRM,ETH,ETHA,...,HD1,HY1,ORI_QU1,TEMP,UTM_E,UTM_N,VEL,HEI,DIA,CP_NOb
0,0,0,0.0,0.0,0.0,0.0,2.400837e+09,0.0,0.0,0.0,...,15.0,5460.0,189.2,202.0,54060.527015,154494.049724,9.1,51.0,0.876163,"[b'A', b'3', b'4', b'0', b'0', b'0', b'4', b'7..."
1,1,1,0.0,0.0,0.0,0.0,1.030931e+09,0.0,0.0,0.0,...,24.0,8736.0,30.0,120.0,54325.725374,154014.654113,7.3,35.0,0.354318,"[b'A', b'3', b'4', b'0', b'0', b'1', b'0', b'9..."
2,2,2,0.0,0.0,0.0,0.0,2.658376e+08,0.0,0.0,0.0,...,24.0,8736.0,30.0,150.0,54179.197179,155026.404876,9.1,53.0,0.329237,"[b'A', b'3', b'4', b'0', b'0', b'1', b'1', b'8..."
3,3,3,0.0,0.0,0.0,0.0,2.356877e+09,0.0,0.0,0.0,...,24.0,8736.0,3.0,150.0,54200.082535,155161.330165,9.1,76.0,0.104114,"[b'A', b'3', b'4', b'0', b'0', b'1', b'7', b'2..."
4,4,4,0.0,0.0,0.0,0.0,8.072385e+08,0.0,0.0,0.0,...,24.0,8736.0,45.0,150.0,54383.208009,154491.108945,9.1,57.0,0.403231,"[b'A', b'3', b'4', b'A', b'1', b'3', b'9', b'2..."


- 控制料堆排放量
  - 將`df`另存成`fth`檔案，釋放記憶體，以接續nc檔案之儲存。

   


In [44]:
#blanck the PY sources
PY=pv.loc[pv.CP_NOb.map(lambda x:x[8:10]==[b'P', b'Y'])]
nPY=len(PY)
a=np.zeros(ntm*nPY)
for t in range(ntm):
  t1,t2=t*nPY,(t+1)*nPY
  a[t1:t2]=t*nopts+np.array(PY.index,dtype=int)
for c in colc:
  ca=df.loc[a,c]/5.
  df.loc[a,c]=ca
df.to_feather('df'+mm+'.fth')
pv.set_index('CP_NO').to_csv('pv'+mm+'.csv')
sys.exit()



## 檔案下載
- `python`程式：[ptseE_ONS.py](https://raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/EmisProc/ptse/ptseE.py)。
- `jupyter-notebook`檔案
  - [ptseE_ONS.ipynb](https://raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/EmisProc/ptse/ptseE.ipynb)
  - [nbviewer](https://nbviewer.org/raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/EmisProc/ptse/ptseE.ipynb)


## Reference
